# 11\. [**Répartition**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/11_dispatching.ipynb)<br/>([*Dispatching*](https://scikit-learn.org/stable/modules/array_api.html))

**Reste** : 2 pages, 0 exemples

# 11.1. [**Prise en charge de l'API Array (expérimental)**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/11_1_array_api.ipynb#array-api-support-experimental) ([*Array API support (experimental)*](https://scikit-learn.org/stable/modules/array_api.html#array-api-support-experimental))

La spécification de l'[API Array](https://data-apis.org/array-api/latest/) définit une API standard pour toutes les bibliothèques de manipulation de tableaux avec une API similaire à NumPy.

Certains estimateurs de scikit-learn qui dépendent principalement de NumPy (par opposition à l'utilisation de Cython) pour implémenter la logique algorithmique de leurs méthodes `fit`, `predict` ou `transform` peuvent être configurés pour accepter des structures de données compatibles avec l'API Array et envoyer automatiquement les opérations vers l'espace de noms sous-jacent au lieu de dépendre de NumPy.

À ce stade, cette prise en charge est considérée comme expérimentale et doit être activée explicitement comme expliqué ci-dessous.

**Note** Actuellement, seules les interfaces `cupy.array_api` et `numpy.array_api` sont connues pour fonctionner avec les estimateurs de scikit-learn.

* 11.1.1. [**Exemple d'utilisation**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/11_1_array_api.ipynb#example-usage) ([*Example usage*](https://scikit-learn.org/stable/modules/array_api.html#example-usage))

* 11.1.2. [**Estimateurs avec prise en charge des entrées compatibles avec l'API Array**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/11_1_array_api.ipynb#estimators-with-support-for-array-api-compatible-inputs) ([*Estimators with support for Array API-compatible inputs*](https://scikit-learn.org/stable/modules/array_api.html#estimators-with-support-for-array-api-compatible-inputs#estimators-with-support-for-array-api-compatible-inputs))

In [6]:
!pip show cupy-cuda12x

Name: cupy-cuda12x
Version: 12.1.0
Summary: CuPy: NumPy & SciPy for GPU
Home-page: https://cupy.dev/
Author: Seiya Tokui
Author-email: tokui@preferred.jp
License: MIT License
Location: C:\Users\franc\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: fastrlock, numpy
Required-by: 


## <a id='example-usage'></a> 11.1.1. Exemple d'utilisation

Voici un exemple de code pour montrer comment utiliser [CuPy](https://cupy.dev/) pour exécuter [**`LinearDiscriminantAnalysis`**](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html#sklearn.discriminant_analysis.LinearDiscriminantAnalysis) sur un GPU :

In [7]:
from sklearn.datasets import make_classification
from sklearn import config_context
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import cupy.array_api as xp

X_np, y_np = make_classification(random_state=0)
X_cu = xp.asarray(X_np)
y_cu = xp.asarray(y_np)
X_cu.device

with config_context(array_api_dispatch=True):
    lda = LinearDiscriminantAnalysis()
    X_trans = lda.fit_transform(X_cu, y_cu)
X_trans.device

C:\Users\franc\AppData\Local\Temp\ipykernel_12172\3949146708.py:4: UserWarning: The cupy.array_api submodule is still experimental. See NEP 47.
  import cupy.array_api as xp


<CUDA Device 0>

Une fois que le modèle est entraîné, les attributs ajustés qui sont des tableaux proviendront également du même espace de noms Array API que les données d'entraînement. Par exemple, si l'espace de noms Array API de CuPy a été utilisé pour l'entraînement, alors les attributs ajustés seront sur le GPU. Nous fournissons une fonction expérimentale `_estimator_with_converted_arrays` qui transfère les attributs d'un estimateur de l'API Array à un ndarray :

In [8]:
from sklearn.utils._array_api import _estimator_with_converted_arrays
cupy_to_ndarray = lambda array: array._array.get()
lda_np = _estimator_with_converted_arrays(lda, cupy_to_ndarray)
X_trans = lda_np.transform(X_np)
type(X_trans)

numpy.ndarray

## <a id='estimators-with-support-for-array-api-compatible-inputs'></a> 11.1.2. Estimateurs prenant en charge les entrées compatibles avec l'API Array

* [**`discriminant_analysis.LinearDiscriminantAnalysis`**](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html#sklearn.discriminant_analysis.LinearDiscriminantAnalysis) (avec `solver="svd"`)

La couverture pour plus d'estimateurs devrait augmenter avec le temps. Veuillez suivre la [meta-issue dédiée sur GitHub](https://github.com/scikit-learn/scikit-learn/issues/22352) pour suivre les progrès.